### Import custom modules from current folder

In [3]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import nltk
from sklearn.model_selection import train_test_split
# from simple_text_representation.classes import Text
# from simple_text_representation.models import Database
import pandas as pd
import numpy as np
# from nltk.draw.tree import draw_trees

In [5]:
# database = Database('educationalTexts', 'postgres', '', '0.0.0.0', 5432)
# path = r'http://localhost/'

In [6]:
def transformToString(text):
    textStr = ''

    for paragraph in text:
        for line in paragraph:
            textStr = textStr + line
    return textStr

### Load the data

In [7]:
dfSeventh = pd.read_csv('../Data/textsSeventhgrade.csv')
dfEighth = pd.read_csv('../Data/textsEighthgrade.csv')
dfNinth = pd.read_csv('../Data/textsNinthgrade.csv')
dfTenth = pd.read_csv('../Data/textsTenthGrade.csv')
dfEleventh = pd.read_csv('../Data/textsEleventhgrade.csv')

### Transform the data to known structures

In [8]:
def transform_csv_to_structure(df):
    text_ids = df['id'].unique()
    texts = list()
    for text_id in text_ids:
        sentence_grouped = df.where(dfSeventh['id'] == 1)
        clean_df = df[pd.notnull(sentence_grouped['id'])]
        paragraphs = list()
        df_paragraphs = clean_df.groupby('paragraph_id').apply(lambda x: "%s" % '|'.join(x['value'])).values
        for df_paragraph in df_paragraphs:
            paragraphs.append(df_paragraph.split('|'))
        texts.append(paragraphs)

    return texts

In [9]:
textOfSeventhGrade = transform_csv_to_structure(dfSeventh)
textOfEightGrade = transform_csv_to_structure(dfEighth)
textOfNineGrade = transform_csv_to_structure(dfNinth)
textOfTenthGrade = transform_csv_to_structure(dfTenth)
textOfEleventhGrade = transform_csv_to_structure(dfEleventh)

textsFormatedSG = [transformToString(textArr) for textArr in textOfSeventhGrade]
textsFormatedEG = [transformToString(textArr) for textArr in textOfEightGrade]
textsFormatedNG = [transformToString(textArr) for textArr in textOfNineGrade]
textsFormatedTG = [transformToString(textArr) for textArr in textOfTenthGrade]
textsFormatedEG = [transformToString(textArr) for textArr in textOfEleventhGrade]

### Format train and test data

In [10]:
data = np.concatenate((np.array(textsFormatedSG),
                       np.array(textsFormatedEG),
                       np.array(textsFormatedNG),
                       np.array(textsFormatedTG),
                       np.array(textsFormatedEG)  )) 
labels = np.concatenate((np.full(len(textsFormatedSG), 0),
                         np.full(len(textsFormatedEG), 1),
                         np.full(len(textsFormatedNG), 2),
                         np.full(len(textsFormatedTG), 3),
                         np.full(len(textsFormatedEG), 4)))

In [11]:
len(data)

179

In [12]:
len(labels)

179

In [13]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [14]:
len(y_train)

143

In [15]:
len(x_train)

143

### Load the trained embeddings

In [16]:
EMBEDDINGS_DIMESION = 300

In [17]:
embeddingsIndex = dict()
f = open('../SBW-vectors-300-min5.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddingsIndex[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddingsIndex))

Loaded 1000654 word vectors.


### Preprocesing the data

In [18]:
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import torch
from math import ceil

# from torch.nn.utils.rnn import pad_sequence
# import spacy
torch.__version__

'0.4.0'

In [19]:
# nlp = spacy.load('es')
# doc = nlp(textOfSeventhGrade[0][0][0])
# token_sentence = [token for token in doc]
# # token_sentence

In [20]:
def createEmbedingMatrix(tokenizer, vocabSize):
    embeddingMatrix = np.zeros((vocabSize, 300))
    for word, i in tokenizer.word_index.items():
        embeddingVector = embeddingsIndex.get(word)
        if embeddingVector is not None:
            embeddingMatrix[i] = embeddingVector
    return embeddingMatrix

In [21]:
def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    vocabSize = len(tokenizer.word_index) + 1
    encodedData = tokenizer.texts_to_sequences(x)
    maxLength = EMBEDDINGS_DIMESION
    paddedData = pad_sequences(encodedData, maxlen=maxLength, padding='post')
    embeddings = createEmbedingMatrix(tokenizer, vocabSize)
    
    return paddedData

In [22]:
def createTrainData(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    vocabSize = len(tokenizer.word_index) + 1
    encodedData = tokenizer.texts_to_sequences(x)
    maxLength = EMBEDDINGS_DIMESION
    paddedData = pad_sequences(encodedData, maxlen=maxLength, padding='post')
    embeddings = createEmbedingMatrix(tokenizer, vocabSize)
    
    return paddedData, embeddings, maxLength, vocabSize

In [68]:
x_train_tokenize, x_train_embeddings, x_train_max_features, x_train_vocab_size  = createTrainData(x_train)
x_test_tokenize = tokenize(x_test)


torch_train_embeddings = torch.from_numpy(x_train_embeddings)
torch_train_tokenize = torch.from_numpy(x_train_tokenize)
torch_train_tokenize = torch_train_tokenize.type(torch.LongTensor)
torch_train_tokenize.req_grad = True
# torch_train_embeddings
# torch_train_tokenize
x_train_embeddings.shape

(341, 300)

### Creating the model

In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [25]:
# embed = torch.ones([300, 1, 500])

In [26]:
# embed.shape

In [27]:
# m = torch.ones([300, 1, 5])

In [28]:
# d = [1, 2, 3, 4, 5]

In [29]:
# "A B C D E" -> 12 13 45 23 12 -> [1.2, 2.3] [1.3, 4.3] [0.2, 2.3] [1.2, 2.3] [1.2, 2.3] 
#                               -> [d, d, d, d, d]

In [30]:
# torch.cat((embed, m), dim=2)

In [31]:
# m.shape

In [32]:
# len(torch.chunk(embed, chunks=300))

In [33]:
# d = torch.chunk(embed, chunks=300)

In [34]:
# for v in d:
#     torch.cat((embed, m),)

In [35]:
# torch.cat((embed, m), dim=2)

In [36]:
def create_weight_matrix(dict_embeddings):
    matrix_len = len(dict_embeddings)
    weights_matrix_emb = np.zeros((matrix_len, EMBEDDINGS_DIMESION))
    
    for i, word in enumerate(dict_embeddings):
        weights_matrix_emb[i] = dict_embeddings[word]
        
    return  weights_matrix_emb

In [37]:
def create_emb_layer(emb_matrix, non_trainable=False):
    num_embeddings, embedding_dim = emb_matrix.shape
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.weight = nn.Parameter(emb_matrix)
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [72]:
class LSTMClassifier(nn.Module):

    def __init__(self, weights_matrix, hidden_dim, vocab_size, batch_size, tagset_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim

        num_embeddings, embedding_dim = weights_matrix.shape
        self.word_embeddings = nn.Embedding(num_embeddings, embedding_dim)
        self.word_embeddings.weight.data.copy_(weights_matrix)
        self.word_embeddings.weight.requires_grad = False

#         self.word_embeddings, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, 1)

        # The linear layer that maps from hidden state space to tag space
        # + 5 in the hidden_dim after the LSTM
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden(batch_size)

    def init_hidden(self, batch_size):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))

    # Recibe el batch
    # [[1,2,3,4,5,6]] [[0.4,14,5123,123,21]]
    # [[]]
    def forward(self, text):
        embeds = self.word_embeddings(text) # batch de textos
        # Append de las metricas al batch ( con cat )
        # torch.cat((embeds, metrics), 2)
        # (seq_len, batch_size, embed_size)
        embeds.requires_grad = True
        print('Embeds', embeds.shape);
        print('Embeds', embeds.view(300, 32, -1).shape);
        print('Hidden Size:', self.hidden[0].shape)
        
        lstm_out, self.hidden = self.lstm(embeds.view(300, 32, -1), self.hidden)
        
        
        tag_space = self.hidden2tag(lstm_out.view(len(text), -1))
        
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

### Training the model

In [39]:
#batch_size - num_hidden_neurons - epochs - embedding_size - precision
testParams2 = [
  [32, 64, 10],
  [32, 128, 10],
  [32, 64, 20],
  [32, 128, 20],
  [32, 256, 20],
  [32, 64, 30],
  [32, 128, 30],
  [32, 64, 40],
  [32, 128, 40],
  [64, 64, 40],
  [64, 128, 40],
  [64, 256, 40],
  [32, 64, 50],
]

#### Transform dictionary into matrix

In [40]:
final_weigth_matrix = torch.from_numpy(create_weight_matrix(embeddingsIndex))

In [69]:
final_weigth_matrix = final_weigth_matrix.type(torch.FloatTensor)
final_weigth_matrix.shape

torch.Size([1000654, 300])

In [42]:
def split_train_batchs(arr_tokens, arr_targets, batch_size):
    max_length = ceil(len(arr_tokens)/batch_size)
    torch_train_batchs = list()
    y_train_batchs = list()

    for i in range(max_length):
        torch_train_batch = arr_tokens[i*batch_size:(i+1)*batch_size]
        y_train_batch = arr_targets[i*batch_size:(i+1)*batch_size]
        torch_train_batchs.append(torch_train_batch)
        y_train_batchs.append(y_train_batch)

    data_train_batch = list(zip(torch_train_batchs, y_train_batchs))

    return data_train_batch

In [43]:
def testModel(hidden_dim, epochs, texts_batch_size, train_batchs):
    model = LSTMClassifier(final_weigth_matrix, hidden_dim, x_train_vocab_size, texts_batch_size, 5)
    loss_function = nn.NLLLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.1)
    
    for epoch in range(epochs):
        for batch_text, targets in train_texts_batchs:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.hidden = model.init_hidden(texts_batch_size)

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Tensors of word indices.
#             sentence_in = prepare_sequence(sentence, word_to_ix)
#             targets = prepare_sequence(tags, tag_to_ix)

            # Step 3. Run our forward pass.
            tag_scores = model(batch_text)
            print('Tag scores: /n', tag_scores)

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            loss = loss_function(tag_scores, targets)
            print('Loss results/n', loss)
            loss.backward()
            optimizer.step()

In [73]:
for param_batch_size, num_hidden_neurons, epochs in testParams2:
    train_texts_batchs = split_train_batchs(torch_train_tokenize, y_train, param_batch_size) # tokens -> 300 x batch_size x num_text/batch_size
    print('Hidden layer:', num_hidden_neurons)
    print('Epochs:', epochs)
    print('Batch size:', param_batch_size)
    testModel(num_hidden_neurons, epochs, param_batch_size, train_texts_batchs)


Hidden layer: 64
Epochs: 10
Batch size: 32
Vocab Size:  341
Embeds torch.Size([32, 300, 300])
Embeds torch.Size([300, 32, 300])
Hidden Size: torch.Size([1, 32, 64])


RuntimeError: size mismatch, m1: [32 x 19200], m2: [64 x 5] at /opt/conda/conda-bld/pytorch_1524590031827/work/aten/src/TH/generic/THTensorMath.c:2033

### Calculate the results of the model